# 장르+줄거리

### 전처리

In [ ]:
!pip install konlpy

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
!pip install sentence_transformers

In [ ]:
import torch
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from urllib.request import urlopen
import os
import re
from konlpy.tag import Okt, Kkma, Komoran
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import networkx
from tqdm.autonotebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from gensim.models import word2vec
import itertools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/캡스톤/코드/캡스톤/데이터/연극/연극_TOP77_장소병합.xlsx')
df = df.drop(df.columns[0], axis=1)
# df = df.drop(df.columns[2], axis=1)
df.head(3)

,Title,Text_clear_num,연극 이름,공연소개,줄거리,장르
0,연극 〈운빨로맨스〉- 대학로,점에 살고 점에 죽는 점보늬의 호랑이띠 남자와 하룻밤 보내기 프로젝트목숨이 걸린 중...,!로맨틱코미디 연극〈운빨로맨스〉! - 대구,NaN,점에 살고 점에 죽는 점보늬의 호랑이띠 남자와 하룻밤 보내기 프로젝트 목숨이 걸린 ...,"로맨스, 코믹, 공감/힐링"
1,(리얼타임 코믹연극) 택시안에서 - 서울,웃음 감동 사랑이 시작되는 리얼타엄 코믹연극 공연시간 투 콩감백배 다양한에피소드와 ...,(리얼타임 코믹연극) 택시안에서 - 부산,NaN,쾌활하고 유쾌한 택시 운전사 민수 그리고 하영과 소희 두 남녀의 운명적인 만남!\n...,"코믹, 로맨스, 공감/힐링"
2,(코믹연극) 달동네-부산,전사 오픈런 부족 초대합니다 개스 테 그를 코 뻔 부 토사 사으소시 서사으 냈 이거...,(코믹연극) 달동네-부산,NaN,"정음의 아버지 경민은 행정착오로 인해 월남전 참전 중 전사자 처리가 되고,........","코믹, 감동, 드라마"


In [ ]:
df1 = df[['Title', '줄거리', '장르']]
df1.head(3)

,Title,줄거리,장르
0,연극 〈운빨로맨스〉- 대학로,점에 살고 점에 죽는 점보늬의 호랑이띠 남자와 하룻밤 보내기 프로젝트 목숨이 걸린 ...,"로맨스, 코믹, 공감/힐링"
1,(리얼타임 코믹연극) 택시안에서 - 서울,쾌활하고 유쾌한 택시 운전사 민수 그리고 하영과 소희 두 남녀의 운명적인 만남!\n...,"코믹, 로맨스, 공감/힐링"
2,(코믹연극) 달동네-부산,"정음의 아버지 경민은 행정착오로 인해 월남전 참전 중 전사자 처리가 되고,........","코믹, 감동, 드라마"


In [ ]:
df1['장르']=df1['장르'].str.split(',')
df1['장르'] = df1['장르'].apply(lambda x : (' ').join(x))
df1.head(3)

<ipython-input-157-64c876e4423b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['장르']=df1['장르'].str.split(',')
<ipython-input-157-64c876e4423b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['장르'] = df1['장르'].apply(lambda x : (' ').join(x))


,Title,줄거리,장르
0,연극 〈운빨로맨스〉- 대학로,점에 살고 점에 죽는 점보늬의 호랑이띠 남자와 하룻밤 보내기 프로젝트 목숨이 걸린 ...,로맨스 코믹 공감/힐링
1,(리얼타임 코믹연극) 택시안에서 - 서울,쾌활하고 유쾌한 택시 운전사 민수 그리고 하영과 소희 두 남녀의 운명적인 만남!\n...,코믹 로맨스 공감/힐링
2,(코믹연극) 달동네-부산,"정음의 아버지 경민은 행정착오로 인해 월남전 참전 중 전사자 처리가 되고,........",코믹 감동 드라마


In [ ]:
df1['줄거리'] = df1['줄거리'].apply(lambda x: re.sub(r'[^가-힣a-zA-Z\s]', '', str(x)))
df1.head(3)

<ipython-input-158-7b1018866824>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['줄거리'] = df1['줄거리'].apply(lambda x: re.sub(r'[^가-힣a-zA-Z\s]', '', str(x)))


,Title,줄거리,장르
0,연극 〈운빨로맨스〉- 대학로,점에 살고 점에 죽는 점보늬의 호랑이띠 남자와 하룻밤 보내기 프로젝트 목숨이 걸린 ...,로맨스 코믹 공감/힐링
1,(리얼타임 코믹연극) 택시안에서 - 서울,쾌활하고 유쾌한 택시 운전사 민수 그리고 하영과 소희 두 남녀의 운명적인 만남\n남...,코믹 로맨스 공감/힐링
2,(코믹연극) 달동네-부산,정음의 아버지 경민은 행정착오로 인해 월남전 참전 중 전사자 처리가 되고\n잘못된 ...,코믹 감동 드라마


In [ ]:
# 줄거리 전체 + 장르 합쳐서 새로운 열 생성
df_data = df1.drop(df1[(df1['장르']=='') | (df1['줄거리'] =='')].index)
df_data['genres_keywords'] = df1['장르'] + " " +(df1['줄거리'])

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/캡스톤/데이터/뮤지컬/뮤지컬_TOP77.xlsx")
df = df.drop(df.columns[0], axis=1)
# df = df.drop(df.columns[2], axis=1)
df.head()

,Title,Text_clear_num,뮤지컬 이름,공연소개,줄거리,장르
0,"2023 〈스웨그에이지 : 외쳐, 조선〉 - 인천",인기 우리의 작은 외침이 세상을 바꾼파 우리의 작은 외침이 세상을 바꾼다 우리들의 ...,"2023 〈스웨그에이지 : 외쳐, 조선〉 - 인천",NaN,시조'가 국가이념인 상상 속의 '조선'\n삶의 고단함과 역경을 시조 속에 담아 훌훌...,"역사극, 판타지"
1,2023 창작뮤지컬어워드 NEXT,창작뮤지컬어워드 충무마트센터가 가능성 있는 참작뮤지컬 발굴을 플랫품인 창작뮤지컬머워...,2023 창작뮤지컬어워드 NEXT,None,헌책방을 운영하는 에이미는 여느 날과 다름없이 책들에게 인사를 건네며 하루를 시작한...,드라마
2,2023 최현우 Answer - 성남,적 서니 정남아트센터 오페라하우스 이하늘이엔티 빅탑엔터테인언트 지라온플레이 년간의 ...,2023 최현우 Answer - 성남,NaN,"마술사로 걸어온 27년\n2,700여회 공연, 200만 명 관객 돌파\n세계가 인정...","마술, 판타지"
3,2023~24 최현우 Answer,마포아트센터 아트홀 맥 외는 이라을레이 이라오 저아아개 년간의 마술 노하우를 집대성...,2023~24 최현우 Answer,NaN,"마술사로 걸어온 27년\n2,700여회 공연, 200만 명 관객 돌파\n세계가 인정...","마술, 판타지"
4,2023송승환의 오리지널 난타(대전),렵 노역 톨 한받때약교야트을좌적매치도 슈 태빼 도시 해외공연투어 장기공연의 힘 오직...,NaN,NaN,세 명의 요리사가 하루 일과를 시작한다. 즐겁게 요리를 준비하는 동안 지배인의 깜짝...,"코믹, 액션"


In [ ]:
df1 = df[['Title', '줄거리', '장르']]
df1.head(3)

,Title,줄거리,장르
0,"2023 〈스웨그에이지 : 외쳐, 조선〉 - 인천",시조'가 국가이념인 상상 속의 '조선'\n삶의 고단함과 역경을 시조 속에 담아 훌훌...,"역사극, 판타지"
1,2023 창작뮤지컬어워드 NEXT,헌책방을 운영하는 에이미는 여느 날과 다름없이 책들에게 인사를 건네며 하루를 시작한...,드라마
2,2023 최현우 Answer - 성남,"마술사로 걸어온 27년\n2,700여회 공연, 200만 명 관객 돌파\n세계가 인정...","마술, 판타지"


In [ ]:
df1['장르']=df1['장르'].str.split(',')
df1['장르'] = df1['장르'].apply(lambda x : (' ').join(x))

df1['줄거리'] = df1['줄거리'].apply(lambda x: re.sub(r'[^가-힣a-zA-Z\s]', '', str(x)))
df_data = df1.drop(df1[(df1['장르']=='') | (df1['줄거리'] =='')].index)
df_data['genres_keywords'] = df1['장르'] + " " +(df1['줄거리'])

<ipython-input-10-7bf060d3e4ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['장르']=df1['장르'].str.split(',')
<ipython-input-10-7bf060d3e4ab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['장르'] = df1['장르'].apply(lambda x : (' ').join(x))
<ipython-input-10-7bf060d3e4ab>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

### 장르 + 줄거리 유사도

In [ ]:
from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer('jhgan/ko-sroberta-multitask')
embedder.max_seq_len = 128

titles = df_data['Title'].tolist()
stories = df_data['genres_keywords'].tolist()

# Fit model to corpus qnd push to GPU
story_embeddings = embedder.encode(stories, convert_to_tensor=True)

In [ ]:
def semantic_search(input_data):
  title_list = []
  story_list = []
  score_list = []

  results = pd.DataFrame()
  # Find the closest 5 stories of the corpus for each query sentence based on cosine similarity
  top_k = min(6, len(story_embeddings))

  # If the input is too short to be a story or its not in the dataset
  if len(input_data) < 20 and input_data not in titles:
    print('Title Not Found')

  # If input is in the dataset
  elif input_data in titles:
    query_embeddings = embedder.encode(str(df_data[df_data['Title'] == input_data]['genres_keywords'])[5:-33], convert_to_tensor=True)

    # Use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embeddings, story_embeddings)[0]
    top_results = torch.topk(cos_scores, k=65)

    input_data_2 = input_data.replace('.', '.\n')
    print("\n\n======================")
    print("    TOP RESULTS")
    print("======================\n")

    # For score, index in torch.topk(cos_scores, k=top_k) use index  locator for feature lists
    # push score to cpu and convert to 1D array
    for score, idx in zip(top_results[0], top_results[1]):
      title_list.append(titles[idx])
      story_list.append(stories[idx])
      score_list.append(score.cpu().numpy().flatten())

    results['Title'] = title_list
    results['Story'] = story_list
    results['Score'] = score_list
    # return dictionary
    return results.iloc[1:, :]

  # If the input is long enough to be a story which is in the dataset
  elif len(input_data) > 20 and input_data not in titles:
    # Find the closest 5 stories of the corpus for each query sentence based on cosine similarity
    query_embeddings = embedder.encode(input_data, convert_to_tensor=True)

    # Use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embeddings, story_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    input_data = input_data.replace('.', '.\n')
    print(input_data)
    print("\n\n======================")
    print("    TOP RESULTS")
    print("======================\n")

    # For score, index in torch.topk(cos_scores, k=top_k) use index  locator for feature lists
    # push score to cpu and convert to 1D array
    for score, idx in zip(top_results[0], top_results[1]):
      title_list.append(titles[idx])
      story_list.append(stories[idx])
      score_list.append(score.cpu().numpy().flatten())

    # Push results to dictionary columns
    results['Title'] = title_list
    results['Story'] = story_list
    results['Score'] = score_list
    # return dictionary
    return results

In [ ]:
### 뮤지컬 장르+줄거리 임베딩
from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer('jhgan/ko-sroberta-multitask')
embedder.max_seq_len = 128

titles = df_data['Title'].tolist()
stories = df_data['genres_keywords'].tolist()

# Fit model to corpus qnd push to GPU
story_embeddings = embedder.encode(stories, convert_to_tensor=True)

In [ ]:
def semantic_search_1(input_data):
  title_list = []
  story_list = []
  score_list = []

  results = pd.DataFrame()
  # Find the closest 5 stories of the corpus for each query sentence based on cosine similarity
  top_k = min(6, len(story_embeddings))

  # If the input is too short to be a story or its not in the dataset
  if len(input_data) < 20 and input_data not in titles:
    print('Title Not Found')

  # If input is in the dataset
  elif input_data in titles:
    query_embeddings = embedder.encode(str(df_data[df_data['Title'] == input_data]['genres_keywords'])[5:-33], convert_to_tensor=True)

    # Use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embeddings, story_embeddings)[0]
    top_results = torch.topk(cos_scores, k=65)

    input_data_2 = input_data.replace('.', '.\n')
    print("\n\n======================")
    print("    TOP RESULTS")
    print("======================\n")

    # For score, index in torch.topk(cos_scores, k=top_k) use index  locator for feature lists
    # push score to cpu and convert to 1D array
    for score, idx in zip(top_results[0], top_results[1]):
      title_list.append(titles[idx])
      story_list.append(stories[idx])
      score_list.append(score.cpu().numpy().flatten())

    results['Title'] = title_list
    results['Story'] = story_list
    results['Score'] = score_list
    # return dictionary
    return results.iloc[1:, :]

  # If the input is long enough to be a story which is in the dataset
  elif len(input_data) > 20 and input_data not in titles:
    # Find the closest 5 stories of the corpus for each query sentence based on cosine similarity
    query_embeddings = embedder.encode(input_data, convert_to_tensor=True)

    # Use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embeddings, story_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    input_data = input_data.replace('.', '.\n')
    print(input_data)
    print("\n\n======================")
    print("    TOP RESULTS")
    print("======================\n")

    # For score, index in torch.topk(cos_scores, k=top_k) use index  locator for feature lists
    # push score to cpu and convert to 1D array
    for score, idx in zip(top_results[0], top_results[1]):
      title_list.append(titles[idx])
      story_list.append(stories[idx])
      score_list.append(score.cpu().numpy().flatten())

    # Push results to dictionary columns
    results['Title'] = title_list
    results['Story'] = story_list
    results['Score'] = score_list
    # return dictionary
    return results

In [ ]:
semantic_search("연극 〈운빨로맨스〉- 대학로")



    TOP RESULTS



,Title,Story,Score
1,뮤직드라마 〈불편한 편의점〉,드라마 공감/힐링 서울역에서 노숙 중인 사내 독고\n어느 날 대 여사 염영숙의 지...,[0.48125947]
2,연극 한뼘사이,로맨스 코믹 한 뼘 더 가까워질 수 있는 사랑의 타이밍을 잡을 수 있을까\n날 좋...,[0.47149545]
3,코믹감동 휴먼판타지 연극 〈2호선 세입자〉,코믹 감동 힐링 우당탕탕 웃음만발 호선\n호선에 사람이 산다고\n아버지의 뒤를 이어...,[0.432207]
4,3대가 웃고 우는 연극 〈염쟁이 유씨〉,감동 공감/힐링 주인공 유씨는 대대로 염을 하던 집안이다 직업에 귀천이 있다하나 ...,[0.43022543]
5,좀비연극 오마이갓,공포 판타지 관객참여형 살려주세요\n경찰에게 걸려온 한 통의 전화 의문의 살인사건이...,[0.42799583]
...,...,...,...
60,연극 〈바닷마을 다이어리〉,드라마 감동 어느 바닷가의 한적한 마을 아버지의 외도로 부모가 떠난 집에 세자매인...,[0.23390073]
61,［2023 SPAF］ 지연X전환 〈에너지_보이지 않는 언어〉,블랙코미디 관객참여형 기후위기를 자신의 언어로 만들 수 있을까\n김지연 전윤환은 ...,[0.19649981]
62,대학로 청소년연극 〈사춘기메들리〉,드라마 코믹 전근이 잦은 아버지의 직장 때문에 친구도 없이 전학을\n다녀야했던 정...,[0.1715059]
63,［2023 SPAF］ 파이브 아트 센터 〈노셔널 히스토리〉,역사극 년 영국으로부터의 독립 이후 바리산 내셔널 정당의 년의 장기 집권과 번의 총...,[0.12111206]


In [ ]:
semantic_search_1("뮤지컬 〈문스토리〉")



    TOP RESULTS



,Title,Story,Score
1,뮤지컬 〈렛미플라이〉,역사극 드라마 아폴로 호가 달을 향해 쏘아 올려진 년의 밤\n동네 최고의 수선장이...,[0.52778757]
2,뮤지컬 〈판〉,코믹 블랙코미디 감동 세기 말 조선 서민들 사이에서 흉흉한 세상을 풍자하는 패관...,[0.4148951]
3,"2023 〈스웨그에이지 : 외쳐, 조선〉 - 인천",역사극 판타지 시조가 국가이념인 상상 속의 조선\n삶의 고단함과 역경을 시조 속에...,[0.4006812]
4,뮤지컬 〈빨래〉,드라마 감동 공감/힐링 서울살이 년 차인 나영과 솔롱고 작가의 꿈을 안고 서울에...,[0.39863056]
5,뮤지컬 〈투모로우 모닝〉,로맨스 공감/힐링 통통 튀는 매력 존 캣의 풋풋함과\n책 캐서린의 무르익은 감...,[0.39050323]
...,...,...,...
60,뮤지컬 〈구텐버그〉,역사극 중세 독일의 작은 마을 슐리머\n포도즙을 짜던 평범한 남자 구텐버그가\n인쇄...,[0.1908222]
61,런던레코드,드라마 감동 런던 외곽에 위치한 낡고 오래된 레코드 샵 주인 존은 오늘도 하루를 ...,[0.16762534]
62,뮤지컬 프리즌,관객 참여형 코믹 락 밴드 연습생들 감옥 가다 가수의 꿈을 안고 혹독한 준비를 해...,[0.15406935]
63,난타(NANTA) - 명동공연,코믹 액션 세 명의 요리사가 하루 일과를 시작한다 즐겁게 요리를 준비하는 동안 지...,[0.1449534]


In [ ]:
a = semantic_search("연극 〈운빨로맨스〉- 대학로")



    TOP RESULTS



In [ ]:
print(type(a))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
selected_columns = a[['Title', 'Score']]
selected_columns['Score'] = selected_columns['Score'].apply(lambda x: x[0])
selected_columns

<ipython-input-21-534d2ed530bf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['Score'] = selected_columns['Score'].apply(lambda x: x[0])


,Title,Score
1,뮤직드라마 〈불편한 편의점〉,0.481259
2,연극 한뼘사이,0.471495
3,코믹감동 휴먼판타지 연극 〈2호선 세입자〉,0.432207
4,3대가 웃고 우는 연극 〈염쟁이 유씨〉,0.430225
5,좀비연극 오마이갓,0.427996
...,...,...
60,연극 〈바닷마을 다이어리〉,0.233901
61,［2023 SPAF］ 지연X전환 〈에너지_보이지 않는 언어〉,0.196500
62,대학로 청소년연극 〈사춘기메들리〉,0.171506
63,［2023 SPAF］ 파이브 아트 센터 〈노셔널 히스토리〉,0.121112


In [ ]:
plot_genre_weight = selected_columns.set_index(keys=['Title'])
plot_genre_weight

,Score
Title,
뮤직드라마 〈불편한 편의점〉,0.481259
연극 한뼘사이,0.471495
코믹감동 휴먼판타지 연극 〈2호선 세입자〉,0.432207
3대가 웃고 우는 연극 〈염쟁이 유씨〉,0.430225
좀비연극 오마이갓,0.427996
...,...
연극 〈바닷마을 다이어리〉,0.233901
［2023 SPAF］ 지연X전환 〈에너지_보이지 않는 언어〉,0.196500
대학로 청소년연극 〈사춘기메들리〉,0.171506


### 별점 가중치 함수


연극 별점가중치

In [ ]:
# 연극_평점
import pandas as pd
play_star = pd.read_excel("/content/drive/MyDrive/캡스톤/데이터/연극/playTop77_star.xlsx")
play_star

,Unnamed: 0,Title,ID,Star,Date
0,5316,연극 〈운빨로맨스〉- 대학로,seeun0***,5,2023.10.10
1,5317,연극 〈운빨로맨스〉- 대학로,isoo9***,5,2023.10.10
2,5318,연극 〈운빨로맨스〉- 대학로,rkgml5***,5,2023.10.09
3,5324,연극 〈운빨로맨스〉- 대학로,da9***,5,2023.10.03
4,5325,연극 〈운빨로맨스〉- 대학로,4322***,5,2023.09.28
...,...,...,...,...,...
9725,9818,그곳에 있었다,judy7***,5,2023.09.14
9726,9819,그곳에 있었다,ddhj***,5,2023.09.03
9727,9820,그곳에 있었다,gustnr4***,5,2023.09.03
9728,9821,그곳에 있었다,pinkhwan***,5,2023.09.03


In [ ]:
print(type(play_star))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
star = play_star['Star']
print(play_star.shape[0])
num_play = play_star.shape[0]
print(type(num_play))

9730
<class 'int'>



코드 설명
1.   1+review_count: 리뷰 수에 1을 더함. 분모가 0이되는 것을 방지하기 위해.
2.   항목 추가



In [ ]:
# user_ratings : 별점리스트, review_count: 전체 행개수
def calculate_weighted_rating (user_ratings, review_count, base_weight=1.0):

  # 가중치 계산
  weight = base_weight*( 1.0 + review_count)    # 사람 수가 적을수록 가중치 감소/ 기본 가중치값 먼저 필요

  # 평균 별점 계산
  average_rating = sum(user_ratings)/len(user_ratings)   # 작품명 같은 것끼리 계산

  # 가중 평균 별점 계산
  weighted_rating = average_rating*weight

  return weighted_rating

## 데이터 함수 적용

*   같은 작품끼리 어떻게 묶을 것인지 정해야함
*   for문 이용해서 작품마다 함수 돌려야함



In [ ]:
# 일단 작품이름 같은 것끼리 묶어야함
# 묶은 것에서 user_rating 리스트 만들기
# 묶은 것 아이템 개수=작품별 리뷰 개수 계산
# for문 이용해서 작품 하나씩 들어가서 돌려야함.
# 결과  --> 작품: 가중치 이런식으로 아웃풋.

In [ ]:
# 제목별로 별점 그룹화
grouped_title = play_star.groupby('Title')['Star'].apply(list).reset_index()
type(grouped_title)

pandas.core.frame.DataFrame

In [ ]:
# 별점 가중치 함수 데이터에 적용

review_count = 100
weighted_ratings = {}

for row in grouped_title.itertuples():
  title = row.Title
  user_ratings = row.Star

  # 가중 평균 계산 함수 호출
  weighted_rating = calculate_weighted_rating(user_ratings, review_count=len(user_ratings))
  weighted_ratings[title] = weighted_rating


for title, weighted_rating in weighted_ratings.items():
    print(f"Title: {title}, Weighted Rating: {weighted_rating}")

Title: (리얼타임 코믹연극) 택시안에서 - 서울, Weighted Rating: 1005.744075829384
Title: (코믹연극) 달동네-부산, Weighted Rating: 73.92857142857143
Title: 10년 연속 1위 연극〈옥탑방고양이〉- 틴틴홀, Weighted Rating: 2462.819607843137
Title: 2023 연극 〈친정엄마와 2박 3일〉 - 고양, Weighted Rating: 33.142857142857146
Title: 2시간탈출 졸탄쇼, Weighted Rating: 653.7720588235295
Title: 3대가 웃고 우는 연극 〈염쟁이 유씨〉, Weighted Rating: 68.92307692307693
Title: 4D공포연극 〈스위치〉, Weighted Rating: 2457.9555555555557
Title: ★평점9.5★ 코미디의맛/쇼미더퍼니, Weighted Rating: 2124.7640449438204
Title: 공포스릴러연극〈두여자〉- 대구, Weighted Rating: 310.0
Title: 공포연극 조각, Weighted Rating: 617.7890625
Title: 공포연극 ［자취］, Weighted Rating: 966.81
Title: 국민 코믹 연극 〈오백에삼십〉 - 대학로 세우아트센터 1관, Weighted Rating: 2800.8966725043783
Title: 그곳에 있었다, Weighted Rating: 57.81818181818182
Title: 나의 장례식에 와줘, Weighted Rating: 73.92857142857143
Title: 대학로 1위 연극 〈쉬어매드니스〉, Weighted Rating: 2432.7607843137253
Title: 대학로 청소년연극 〈사춘기메들리〉, Weighted Rating: 716.9444444444445
Title: 로맨틱코미디 〈슬기로운 신혼생활〉, Weighted Rating: 533.89814814

In [ ]:
def star_weight(excel):
  import pandas as pd
  star_df = pd.read_excel(excel)
  star = star_df['Star']
  num_play = star_df.shape[0]

  # 일단 작품이름 같은 것끼리 묶어야함
  # 묶은 것에서 user_rating 리스트 만들기
  # 묶은 것 아이템 개수=작품별 리뷰 개수 계산
  # for문 이용해서 작품 하나씩 들어가서 돌려야함.
  # 결과  --> 작품: 가중치 이런식으로 아웃풋.

  # 제목별로 별점 그룹화
  grouped_title = star_df.groupby('Title')['Star'].apply(list).reset_index()
  type(grouped_title)

  # 별점 가중치 함수 데이터에 적용

  review_count = 100
  weighted_ratings = {}

  for row in grouped_title.itertuples():
    title = row.Title
    user_ratings = row.Star

    # 가중 평균 계산 함수 호출
    weighted_rating = calculate_weighted_rating(user_ratings, review_count=len(user_ratings))
    weighted_ratings[title] = weighted_rating


  for title, weighted_rating in weighted_ratings.items():
      print(f"Title: {title}, Weighted Rating: {weighted_rating}")
  return weighted_ratings

In [ ]:
play_weighted_ratings = star_weight("/content/drive/MyDrive/캡스톤/데이터/연극/playTop77_star.xlsx")
musical_weighted_ratings = star_weight("/content/drive/MyDrive/캡스톤/데이터/뮤지컬/musicalTop77_star.xlsx")

Title: (리얼타임 코믹연극) 택시안에서 - 서울, Weighted Rating: 1005.744075829384
Title: (코믹연극) 달동네-부산, Weighted Rating: 73.92857142857143
Title: 10년 연속 1위 연극〈옥탑방고양이〉- 틴틴홀, Weighted Rating: 2462.819607843137
Title: 2023 연극 〈친정엄마와 2박 3일〉 - 고양, Weighted Rating: 33.142857142857146
Title: 2시간탈출 졸탄쇼, Weighted Rating: 653.7720588235295
Title: 3대가 웃고 우는 연극 〈염쟁이 유씨〉, Weighted Rating: 68.92307692307693
Title: 4D공포연극 〈스위치〉, Weighted Rating: 2457.9555555555557
Title: ★평점9.5★ 코미디의맛/쇼미더퍼니, Weighted Rating: 2124.7640449438204
Title: 공포스릴러연극〈두여자〉- 대구, Weighted Rating: 310.0
Title: 공포연극 조각, Weighted Rating: 617.7890625
Title: 공포연극 ［자취］, Weighted Rating: 966.81
Title: 국민 코믹 연극 〈오백에삼십〉 - 대학로 세우아트센터 1관, Weighted Rating: 2800.8966725043783
Title: 그곳에 있었다, Weighted Rating: 57.81818181818182
Title: 나의 장례식에 와줘, Weighted Rating: 73.92857142857143
Title: 대학로 1위 연극 〈쉬어매드니스〉, Weighted Rating: 2432.7607843137253
Title: 대학로 청소년연극 〈사춘기메들리〉, Weighted Rating: 716.9444444444445
Title: 로맨틱코미디 〈슬기로운 신혼생활〉, Weighted Rating: 533.89814814

### 배우 가중치 함수

#### PlayDB 누적조회순 랭킹 불러오기

---

 - actor_dic 딕셔너리 구조

{ 배우 : { 'index': 순위, 'masterpiece' : [최근공연 3개 리스트] } }




In [ ]:
import pandas as pd
def actor_lanking(csv):
  actor_csv = pd.read_csv(csv, encoding='utf-8')
  actor_csv.columns = ["index", "actor_name", "masterpiece"]

  # 랭킹에 있는 배우들의 작품만 따로 추출 (작품명이 \n으로 이어진 하나의 문자열이기 때문에, 리스트화 시킬 예정)
  actor_mp = actor_csv['masterpiece'].to_list()

  actor_dic = actor_csv.set_index('index').T.to_dict()  # 딕셔너리 형태로 변환

  # key: actor_name, value: index, masterpiece
  for key, value in actor_dic.items():
    temp = value['masterpiece']
    temp2 = temp.split('\n')
    value['masterpiece'] = temp2
  return actor_dic

In [ ]:
play_actor_dic = actor_lanking("/content/drive/MyDrive/캡스톤/데이터/연극/연극 배우 랭킹.csv")
play_actor_dic

In [ ]:
mus_actor_dic = actor_lanking("/content/drive/MyDrive/캡스톤/데이터/뮤지컬/뮤지컬 배우 랭킹.csv")
mus_actor_dic

#### 인터파크 캐스팅 배우 불러오기


---

- inter_dic 딕셔너리 구조

{ 작품 제목 : { 배우 : [캐스팅 배우 리스트] } }

In [ ]:
def cast_actor(excel):
  inter_actor = pd.read_excel(excel)
  inter_actor = inter_actor[['Title', 'cast_act']]

  # 문자열 형태로 되어 있는 cast_act 열을 리스트화 시켜서 다시 집어 넣을 예정
  inter_actor_cast = inter_actor['cast_act']

  inter_list = []
  for i in range(len(inter_actor)):
    string = inter_actor_cast[i]
    ch_string = string.split(', ')
    inter_list.append(ch_string)

  # Title 기준 딕셔너리화
  inter_dic = inter_actor.set_index('Title').T.to_dict()

  # key: Title, cast_act
  i = 0
  for key, value in inter_dic.items():
    value['cast_act'] = inter_list[i]
    i+=1
  return inter_dic, inter_actor

In [ ]:
play_inter_dic, play_inter_actor = cast_actor("/content/drive/MyDrive/캡스톤/데이터/연극/playTop77_casting.xlsx")
play_inter_dic

In [ ]:
play_inter_actor

,Title,cast_act
0,연극 〈운빨로맨스〉- 대학로,"김세울, 문재웅, 변가온, 한선우, 김지후, 장은서, 조은진, 최지선, 박병훈, 신..."
1,(리얼타임 코믹연극) 택시안에서 - 서울,"김로언, 전청일, 한상웅, 김나온, 한재우, 오하성, 이채연, 김진화, 서예선"
2,(코믹연극) 달동네-부산,"조영준, 안애린, 한서아, 류현주, 양채은, 정유나, 곽희규, 김영신, 신희진, 이..."
3,10년 연속 1위 연극〈옥탑방고양이〉- 틴틴홀,"임채민, 정지호, 문지원, 정승지, 양시환, 황민환, 최지환, 장진호, 박지민, 심..."
4,2023 연극 〈친정엄마와 2박 3일〉 - 고양,해당없음
...,...,...
60,공포스릴러연극〈두여자〉- 대구,"김미영, 김린아, 이다은, 엄영실, 이상아, 임성희, 이도윤, 최승환, 문다현, 박..."
61,모든 날 모든 순간,해당없음
62,나의 장례식에 와줘,해당없음
63,3대가 웃고 우는 연극 〈염쟁이 유씨〉,해당없음


In [ ]:
mus_inter_dic, mus_inter_actor = cast_actor("/content/drive/MyDrive/캡스톤/데이터/뮤지컬/musicalTop77_casting.xlsx")
mus_inter_dic

In [ ]:
mus_inter_actor

,Title,cast_act
0,"2023 〈스웨그에이지 : 외쳐, 조선〉 - 인천",해당없음
1,2023 창작뮤지컬어워드 NEXT,해당없음
2,2023 최현우 Answer - 성남,최현우
3,2023~24 최현우 Answer,최현우
4,2023송승환의 오리지널 난타(대전),해당없음
...,...,...
60,뮤지컬 썸데이,"김여진, 서찬양, 김아름, 이채림, 구문회, 이제성, 김선오, 윤동환, 박소진, 조..."
61,뮤지컬 프리즌,"손동진, 최승환, 홍차민, 안중현, 김세훈, 신연우, 하준석, 이강오, 김동훈, 추..."
62,뮤직드라마 당신만이,"장인혁, 박민승, 박준성, 이일진, 손예슬, 이민지, 박세웅, 차은진, 권성민, 이경수"
63,뮤지컬 〈투모로우 모닝〉,"렉스, 정현준, 천관우, 김안젤라, 박혜원, 한수민, 문준혁, 임창민, 오태후, 이..."


#### 배우명 비교
* actor_dic : playDB 랭킹별 최근공연 딕셔너리
* inter_dic : 인터파크 작품별 캐스팅 배우 딕셔너리
---
> 찾는 배우 이름(인터파크 캐스팅 배우)이 actor_dic에 있을 경우, 해당 배우의 랭킹 도출

→ 해당 배우의 랭킹(인덱스)를 활용하여 최근 공연 리스트 불러오기

→ 최근 공연 리스트와 현재 찾는 배우의 공연 이름 비교



In [ ]:
# 프로토타입 (이건 playDB 동명이인이 있을 경우)
res = []
for key, value in play_actor_dic.items():
  if ('최성희'.find(value['actor_name']) > -1):
    res.append(key)
res

[0]

In [ ]:
# 배우 이름 찾기 (랭킹 도출) 함수
def find_value(dictionary, finding_value):
    for key, value in dictionary.items():
        if (finding_value.find(value['actor_name']) > -1):
            return key


In [ ]:
def search_actor(inter_dic, actor_dic):
  actor_mp = {}
  i = 0
  for key1, value1 in inter_dic.items():
      for name in value1['cast_act']:
          index = find_value(actor_dic, name)
          if index is not None:
              for key2, value2 in actor_dic.items():
                  if key2 == index:
                    # 여기서부터 짰음
                      recent_performances = value2['masterpiece']

                      for title in recent_performances:
                        title = title.split('(')
                        title = title[0]

                        # 인터파크 제목에서 최근 공연을 찾음
                        temp = key1.find(title)
                        if temp > -1:
                          print(f"배우 {name}의 최근 공연 '{key1}'이 인터파크 제목에 포함됨.")
                          actor_mp[i] = {'actor' : name, 'Title' : key1}
                          i+=1
  return actor_mp

### 반복문 구조

* for key1, value1 in inter_dic.items(): 인터파크 캐스팅 배우 딕셔너리를 key, value로 쪼갬

* for name in value1['cast_act']: 작품별 캐스팅배우의 이름 기준으로 반복

* index = find_value(actor_dic, name) : 배우 이름 찾기 함수로 배우 찾아서 랭킹을 인덱스에 저장

* if (index!=None): 인덱스가 None이면 랭킹에 없다는 뜻

* for key2, value2 in actor_dic.items(): 배우 랭킹 딕셔너리를 key, value로 쪼갬
        
* if (value2['index'] == index): 랭킹(인덱스)이 같다면

* print(value2['masterpiece']) : 작품명 출력 (여기부터 바꾸면 될 듯)



---


* 연극 배우

In [ ]:
PSA = search_actor(play_inter_dic, play_actor_dic)

배우 박은영의 최근 공연 '뮤직드라마 〈불편한 편의점〉'이 인터파크 제목에 포함됨.
배우 최덕문의 최근 공연 '연극 〈기형도 플레이〉'이 인터파크 제목에 포함됨.
배우 박호산의 최근 공연 '연극 〈기형도 플레이〉'이 인터파크 제목에 포함됨.


In [ ]:
MSA = search_actor(mus_inter_dic, mus_actor_dic)

배우 이재환의 최근 공연 '뮤지컬 〈22년 2개월〉'이 인터파크 제목에 포함됨.
배우 정민의 최근 공연 '뮤지컬 〈구텐버그〉'이 인터파크 제목에 포함됨.
배우 김지현의 최근 공연 '뮤지컬 〈그날들〉 10주년 기념 공연 - 대전'이 인터파크 제목에 포함됨.
배우 린아의 최근 공연 '뮤지컬 〈레미제라블〉 - 부산'이 인터파크 제목에 포함됨.
배우 정민의 최근 공연 '뮤지컬 〈스모크〉'이 인터파크 제목에 포함됨.
배우 김신의의 최근 공연 '뮤지컬 〈삼총사〉'이 인터파크 제목에 포함됨.
배우 김신의의 최근 공연 '뮤지컬 〈삼총사〉'이 인터파크 제목에 포함됨.
배우 송원근의 최근 공연 '뮤지컬 〈오페라의 유령〉 - 서울'이 인터파크 제목에 포함됨.
배우 김아선의 최근 공연 '뮤지컬 〈오페라의 유령〉 - 서울'이 인터파크 제목에 포함됨.
배우 차지연의 최근 공연 '뮤지컬 〈컴프롬어웨이〉'이 인터파크 제목에 포함됨.
배우 문혜원의 최근 공연 '뮤지컬 〈타오르는 어둠 속에서〉'이 인터파크 제목에 포함됨.
배우 송상은의 최근 공연 '뮤지컬 〈블랙메리포핀스〉'이 인터파크 제목에 포함됨.
배우 규현의 최근 공연 '뮤지컬 ＇벤허＇'이 인터파크 제목에 포함됨.
배우 이희정의 최근 공연 '뮤지컬 ＇벤허＇'이 인터파크 제목에 포함됨.
배우 이자람의 최근 공연 '창작가무극 〈순신〉'이 인터파크 제목에 포함됨.
배우 고미경의 최근 공연 '창작가무극 〈순신〉'이 인터파크 제목에 포함됨.


In [ ]:
for key1, value1 in play_inter_dic.items():
    for name in value1['cast_act']:
        index = find_value(play_actor_dic, name)
        if index is not None:
            for key2, value2 in play_actor_dic.items():
                if key2 == index:
                  # 여기서부터 짰음
                    recent_performances = value2['masterpiece']

                    for title in recent_performances:
                      title = title.split('(')
                      title = title[0]

                      # 인터파크 제목에서 최근 공연을 찾음
                      temp = key1.find(title)
                      if temp > -1:
                        print(f"배우 {name}의 최근 공연 '{key1}'이 인터파크 제목에 포함됨.")

    #print(name + " : " + str(index))

배우 박은영의 최근 공연 '뮤직드라마 〈불편한 편의점〉'이 인터파크 제목에 포함됨.
배우 최덕문의 최근 공연 '연극 〈기형도 플레이〉'이 인터파크 제목에 포함됨.
배우 박호산의 최근 공연 '연극 〈기형도 플레이〉'이 인터파크 제목에 포함됨.




---

* 뮤지컬 배우

In [ ]:
for key1, value1 in mus_inter_dic.items():
    for name in value1['cast_act']:
        index = find_value(mus_actor_dic, name)
        if index is not None:
            for key2, value2 in mus_actor_dic.items():
                if key2 == index:
                  # 여기서부터 짰음
                    recent_performances = value2['masterpiece']

                    for title in recent_performances:
                      title = title.split('(')
                      title = title[0]

                      # 인터파크 제목에서 최근 공연을 찾음
                      temp = key1.find(title)
                      if temp > -1:
                        print(f"배우 {name}의 최근 공연 '{key1}'이 인터파크 제목에 포함됨.")

    #print(name + " : " + str(index))

배우 이재환의 최근 공연 '뮤지컬 〈22년 2개월〉'이 인터파크 제목에 포함됨.
배우 정민의 최근 공연 '뮤지컬 〈구텐버그〉'이 인터파크 제목에 포함됨.
배우 김지현의 최근 공연 '뮤지컬 〈그날들〉 10주년 기념 공연 - 대전'이 인터파크 제목에 포함됨.
배우 린아의 최근 공연 '뮤지컬 〈레미제라블〉 - 부산'이 인터파크 제목에 포함됨.
배우 정민의 최근 공연 '뮤지컬 〈스모크〉'이 인터파크 제목에 포함됨.
배우 김신의의 최근 공연 '뮤지컬 〈삼총사〉'이 인터파크 제목에 포함됨.
배우 김신의의 최근 공연 '뮤지컬 〈삼총사〉'이 인터파크 제목에 포함됨.
배우 송원근의 최근 공연 '뮤지컬 〈오페라의 유령〉 - 서울'이 인터파크 제목에 포함됨.
배우 김아선의 최근 공연 '뮤지컬 〈오페라의 유령〉 - 서울'이 인터파크 제목에 포함됨.
배우 차지연의 최근 공연 '뮤지컬 〈컴프롬어웨이〉'이 인터파크 제목에 포함됨.
배우 문혜원의 최근 공연 '뮤지컬 〈타오르는 어둠 속에서〉'이 인터파크 제목에 포함됨.
배우 송상은의 최근 공연 '뮤지컬 〈블랙메리포핀스〉'이 인터파크 제목에 포함됨.
배우 규현의 최근 공연 '뮤지컬 ＇벤허＇'이 인터파크 제목에 포함됨.
배우 이희정의 최근 공연 '뮤지컬 ＇벤허＇'이 인터파크 제목에 포함됨.
배우 이자람의 최근 공연 '창작가무극 〈순신〉'이 인터파크 제목에 포함됨.
배우 고미경의 최근 공연 '창작가무극 〈순신〉'이 인터파크 제목에 포함됨.


### 배우 가중치

* 연극 배우 가중치

* 동명이인 제거 전 : 702명

* 현재는 동명이인의 경우 이름1, 이름2 와 같이 개별적으로 구분
* 만약 이게 필요없다면, 원래 원본 파일을 가지고 실행하면 됨

In [ ]:
def actor_name_dic(actor_dic, inter_dic, SA):
  dic = {}
  for key1, value1 in actor_dic.items():
    dic[key1] = value1['actor_name']


  index = 100  # 순위권 이후 배우 추가용
  for key1, value1 in inter_dic.items():
    for i in value1['cast_act']:  # 공연별 캐스팅 배우 딕셔너리화
      ret = 0 # 순위권 배우 이름 중복 추가 방지용
      for key2, value2 in SA.items(): # 순위권 배우 판별
        if (value2['Title'] == key1 and value2['actor'] == i) or (i == '해당없음'): # 순위권 배우거나, 배우명이 없는 경우 추가 제외
          ret = 1
          break
      if(ret == 0):
        for ran in range(100, index):  # 동명이인 찾기
          if(dic[ran] == i ):
            ret = 1
            break
          else:
            ret = 0

      if(ret == 0): # 그 외 전부 추가
        dic[index] = i
        index += 1
  dic_t = {v:k for k,v in dic.items()}
  return dic, dic_t

In [ ]:
pt_actor_dic, pt_actor_dic_t = actor_name_dic(play_actor_dic, play_inter_dic, PSA)

* 랭킹 배우의 최근 공연작 중 데이터에 해당하는 것이 있다면, 배우 이름 -> 인덱스

In [ ]:
def change_index(search, dic):
  ret_dic = {}
  for key, value in search.items():
    index = dic[value['actor']]
    ret_dic[index] = value['Title']
  return ret_dic

* 모든 배우의 이름을 인덱스로 교체 + 공연별 배우 리스트 생성

In [ ]:
def cast_list(dic1, dic2, dic_t):
  cast_dic = {}
  for key1, value1 in dic2.items():
    cast_act = value1['cast_act']
    cast_index = []
    # 배우 인덱스 저장
    for name in cast_act:
      cast_index.append(dic_t.get(name))
    cast_dic[key1] = cast_index
  return cast_dic

* 배우의 가중치 구하는 함수
  * 사용자가 입력한 공연의 배우가 다른 공연에 있을 경우 +1점
  * 유명한 배우의 공연은 무조건 +2.5점

* 나중에 별점 등의 가중치들과 더하거나 곱할 예정

In [ ]:
def actor_rate(cast_dic, title_name, ci):
  score_list = []
  cast_list = cast_dic[title_name]
  for key, value in cast_dic.items():
    score = 0
    for index in cast_list:
      # 이 공연의 배우가 있을 경우 추천점수 +1점
      if((key != title_name) and (index in value)):
        score += 1
      # 유명한 배우가 있을 경우 추천점수 +2.5점
    for name, title in ci.items():
      if(key == title):
        score += 2.5
    score_list.append(score)
  return score_list

In [ ]:
play_ci = change_index(PSA, pt_actor_dic_t) # 랭킹배우의 이름->인덱스
play_ci

{51: '뮤직드라마 〈불편한 편의점〉', 21: '연극 〈기형도 플레이〉', 3: '연극 〈기형도 플레이〉'}

In [ ]:
play_cast_list = cast_list(pt_actor_dic, play_inter_dic, pt_actor_dic_t)
score_list = actor_rate(play_cast_list, '진짜나쁜소녀', play_ci)
play_inter_actor['actor_score'] = score_list
play_inter_actor.sort_values(by=['actor_score'], ascending=False)

,Title,cast_act,actor_score
18,연극 〈기형도 플레이〉,"최덕문, 우현주, 이석준, 박호산, 이창훈, 이동하, 이은, 김세영, 김승은",5.0
15,뮤직드라마 〈불편한 편의점〉,"장우진, 김동박, 이강혁, 박은영, 조영임, 문상희, 정예지, 정이수, 서지영, 한...",2.5
37,행오버,"최선량, 정현규, 조영광, 신재명, 손준표, 윤건웅, 최창빈, 김민우, 이도희, 이...",1.0
59,로맨틱코미디 〈슬기로운 신혼생활〉,"오세영, 정대경, 김은지, 이슬기, 한재우, 서동현, 김세나, 이재희, 송민주, 이...",1.0
28,연극 한뼘사이,"윤주희, 강단비, 주하영, 유이주, 박정윤, 최하은, 김시윤, 김현지, 김종훈, 정...",1.0
...,...,...,...
27,연극 죽여주는 이야기,"남경화, 노진욱, 이연승, 최다운, 조웅희",0.0
29,연극 핫식스,"이대우, 정회형, 김호준, 김예지, 송다영, 김은수, 임종철, 김동섭, 김정원, 전상혁",0.0
30,연극〈늘근도둑이야기〉,"노진원, 전재형, 박철민, 태항호, 김상묵, 주우성, 유일한, 이호연, 안태영",0.0
31,이머시브씨어터 카지노,"안지현, 남지현, 문수민, 오자송, 최지우, 김유리, 박상혁, 장영웅, 박서연, 안...",0.0




---


* 뮤지컬 배우 가중치

In [ ]:
mt_actor_dic, mt_actor_dic_t = actor_name_dic(mus_actor_dic, mus_inter_dic, MSA)

In [ ]:
mus_ci = change_index(MSA, mt_actor_dic_t) # 랭킹배우의 이름->인덱스
mus_ci

{36: '뮤지컬 〈22년 2개월〉',
 9: '뮤지컬 〈스모크〉',
 50: '뮤지컬 〈그날들〉 10주년 기념 공연 - 대전',
 20: '뮤지컬 〈레미제라블〉 - 부산',
 47: '뮤지컬 〈삼총사〉',
 13: '뮤지컬 〈오페라의 유령〉 - 서울',
 29: '뮤지컬 〈오페라의 유령〉 - 서울',
 2: '뮤지컬 〈컴프롬어웨이〉',
 15: '뮤지컬 〈타오르는 어둠 속에서〉',
 32: '뮤지컬 〈블랙메리포핀스〉',
 5: '뮤지컬 ＇벤허＇',
 25: '뮤지컬 ＇벤허＇',
 11: '창작가무극 〈순신〉',
 99: '창작가무극 〈순신〉'}

In [ ]:
mus_cast_list = cast_list(mt_actor_dic, mus_inter_dic, mt_actor_dic_t)
score_list = actor_rate(mus_cast_list, '뮤지컬 〈오페라의 유령〉 - 서울', mus_ci)
mus_inter_actor['actor_score'] = score_list
mus_inter_actor.sort_values(by=['actor_score'], ascending=False)

,Title,cast_act,actor_score
38,뮤지컬 〈오페라의 유령〉 - 서울,"조승우, 최재림, 김주택, 전동석, 손지수, 송은혜, 송원근, 황건하, 윤영석, 이...",5.0
51,뮤지컬 ＇벤허＇,"박은태, 신성록, 규현, 이지훈, 박민성, 서경수, 윤공주, 이정화, 최지혜, 이정...",5.0
54,창작가무극 〈순신〉,"형남희, 최인형, 이자람, 윤제원, 권성찬, 송문선, 고미경, 금승훈, 김백현, 김...",5.0
21,뮤지컬 〈레미제라블〉 - 부산,"민우혁, 최재림, 김우형, 카이, 조정은, 린아, 임기홍, 육현욱, 박준면, 김영주...",3.5
35,뮤지컬 〈스모크〉,"김재범, 정민, 김경수, 박정원, 강찬, 손유동, 홍승안, 김지유, 김청아, 장보람...",2.5
...,...,...,...
28,뮤지컬 〈문스토리〉,"정상윤, 성연, 김진욱, 김준호, 소정화, 박새힘, 주다온, 장보람, 강찬, 김리현...",0.0
29,뮤지컬 〈빨래〉,해당없음,0.0
30,쇼 뮤지컬 〈시스터즈 (SheStars!)〉,"유연, 신의정, 선민, 황성현, 김려원, 하유진, 정유지, 이예은, 이서영, 정연,...",0.0
31,뮤지컬 〈비더슈탄트〉,"송유택, 안지환, 황순종, 정백선, 김바다, 김지온, 동현, 이진우, 김이담, 손지...",0.0


# 가중치 총합 함수

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0,5))
def weight_calc(weighted_ratings):
  weighted_df = pd.DataFrame(weighted_ratings, index = ['StarScore']).T
  weighted_df[:] = scaler.fit_transform(weighted_df[:])
  return weighted_df

In [ ]:
def castscore_calc(title, actor_dic, inter_dic, actor_dic_t, ci, inter_actor):
  casting_list = cast_list(actor_dic, inter_dic, actor_dic_t)
  score_list = actor_rate(casting_list, title, ci)
  inter_actor['actor_score'] = score_list
  casting = inter_actor.sort_values(by=['actor_score'], ascending=False)
  casting = casting[['Title', 'actor_score']]
  casting = casting.reset_index(drop=True)

  cast = {}
  for i in range(len(casting)):
    key = casting['Title'][i]
    value = casting['actor_score'][i]
    cast[key] = value
    castscore_df = pd.DataFrame(cast, index = ['CastScore']).T
    castscore_df[:] = scaler.fit_transform(castscore_df[:])

  return castscore_df

In [ ]:
## 연극
def plot_genre_calc(title):
  plot_genre = semantic_search(title)
  if plot_genre is not None:
    plot_genre_weight = plot_genre[['Title', 'Score']]
    plot_genre_weight = plot_genre_weight.set_index(keys=['Title'])
    plot_genre_weight[:] = scaler.fit_transform(plot_genre_weight[:])

    return plot_genre_weight
  else:
    # plot_genre가 None인 경우에 대한 처리
    print("검색 결과를 찾을 수 없습니다.")
    return None

In [ ]:
## 뮤지컬
def plot_genre_calc_1(title):
  plot_genre = semantic_search_1(title)
  if plot_genre is not None:
    plot_genre_weight = plot_genre[['Title', 'Score']]
    plot_genre_weight = plot_genre_weight.set_index(keys=['Title'])
    plot_genre_weight[:] = scaler.fit_transform(plot_genre_weight[:])

    return plot_genre_weight
  else:
    # plot_genre가 None인 경우에 대한 처리
    print("검색 결과를 찾을 수 없습니다.")
    return None

In [ ]:
def merge_calc(weighted_df, castscore_df, plot_genre_df, pm = 0):
  func_list = []
  play_w = pd.concat([weighted_df, castscore_df, plot_genre_df], axis=1, join='inner')
  for row in play_w.iterrows():
    #print(row[1]['Weight'])
    ss = row[1]['StarScore']
    cs = row[1]['CastScore']
    pg = row[1]['Score']
    if (pm == 0): # 연극일 경우
      func = pg * 0.65 + cs * 0.2 + ss * 0.15
    else:
      func = cs * 0.35 + pg * 0.5 + ss * 0.15
    func_list.append([row[0], func])
    print(f"Title: {row[0]}, Weighted Rating: {func}")
  return func_list

In [ ]:
def play_score(title):
  weighted_df = weight_calc(play_weighted_ratings)
  castscore_df = castscore_calc(title, pt_actor_dic, play_inter_dic, pt_actor_dic_t, play_ci, play_inter_actor)
  plot_genre_df = plot_genre_calc(title)
  func_list = merge_calc(weighted_df, castscore_df, plot_genre_df)
  play_score = pd.DataFrame(func_list, columns=['Title', 'Score']).sort_values('Score', ascending=False)
  return play_score

In [ ]:
def musical_score(title):
  weighted_df = weight_calc(musical_weighted_ratings)
  castscore_df = castscore_calc(title, mt_actor_dic, mus_inter_dic, mt_actor_dic_t, mus_ci, mus_inter_actor)
  plot_genre_df = plot_genre_calc_1(title)
  func_list = merge_calc(weighted_df, castscore_df, plot_genre_df, pm=1)
  musical_score = pd.DataFrame(func_list, columns=['Title', 'Score']).sort_values('Score', ascending=False)
  return musical_score

In [ ]:
title = "연극〈늘근도둑이야기〉"
play_score(title).head(5)

In [ ]:
title = "연극 〈운빨로맨스〉- 대학로"
play_score(title).head(5)



    TOP RESULTS

Title: (리얼타임 코믹연극) 택시안에서 - 서울, Weighted Rating: 2.499120335512434
Title: (코믹연극) 달동네-부산, Weighted Rating: 1.982047020947842
Title: 10년 연속 1위 연극〈옥탑방고양이〉- 틴틴홀, Weighted Rating: 3.3087579670730394
Title: 2023 연극 〈친정엄마와 2박 3일〉 - 고양, Weighted Rating: 1.2483347553589663
Title: 2시간탈출 졸탄쇼, Weighted Rating: 2.731317904666396
Title: 3대가 웃고 우는 연극 〈염쟁이 유씨〉, Weighted Rating: 2.835777858160527
Title: 4D공포연극 〈스위치〉, Weighted Rating: 2.967097269777567
Title: ★평점9.5★ 코미디의맛/쇼미더퍼니, Weighted Rating: 2.2095523251012135
Title: 공포스릴러연극〈두여자〉- 대구, Weighted Rating: 1.7630255170007418
Title: 공포연극 조각, Weighted Rating: 2.2751292306127953
Title: 공포연극 ［자취］, Weighted Rating: 2.953944270954687
Title: 국민 코믹 연극 〈오백에삼십〉 - 대학로 세우아트센터 1관, Weighted Rating: 2.5863855093323953
Title: 그곳에 있었다, Weighted Rating: 2.6987960219775773
Title: 나의 장례식에 와줘, Weighted Rating: 1.6317755006394208
Title: 대학로 1위 연극 〈쉬어매드니스〉, Weighted Rating: 2.76535849909313
Title: 대학로 청소년연극 〈사춘기메들리〉, Weighted Rating: 0.8262890378334542
Title

,Title,Score
18,뮤직드라마 〈불편한 편의점〉,3.987176
29,연극 한뼘사이,3.802128
36,코믹감동 휴먼판타지 연극 〈2호선 세입자〉,3.444158
21,연극 〈기형도 플레이〉,3.310620
2,10년 연속 1위 연극〈옥탑방고양이〉- 틴틴홀,3.308758


In [ ]:
title = "뮤지컬 〈문스토리〉"
musical_score(title).head(5)



    TOP RESULTS

Title: 난타(NANTA) - 명동공연, Weighted Rating: 0.2366255975405214
Title: 더 맨 얼라이브 〈초이스〉, Weighted Rating: 1.0765398333193374
Title: 뮤지컬 〈22년 2개월〉, Weighted Rating: 2.55531217624176
Title: 뮤지컬 〈곤 투모로우〉, Weighted Rating: 1.1359825573939764
Title: 뮤지컬 〈구텐버그〉, Weighted Rating: 1.0431579213652271
Title: 뮤지컬 〈김종욱 찾기〉, Weighted Rating: 0.9118863681119965
Title: 뮤지컬 〈레미제라블〉 - 부산, Weighted Rating: 2.10053726742386
Title: 뮤지컬 〈레베카〉 10주년 기념공연, Weighted Rating: 2.028147420627959
Title: 뮤지컬 〈렛미플라이〉, Weighted Rating: 2.9954398025397153
Title: 뮤지컬 〈멤피스〉, Weighted Rating: 1.8490205148001206
Title: 뮤지컬 〈블랙메리포핀스〉, Weighted Rating: 2.696585346598388
Title: 뮤지컬 〈비더슈탄트〉, Weighted Rating: 1.0696236015468994
Title: 뮤지컬 〈사칠〉, Weighted Rating: 1.3781115205346977
Title: 뮤지컬 〈삼총사〉, Weighted Rating: 1.814248891336353
Title: 뮤지컬 〈셜록홈즈 : 앤더슨가의 비밀〉, Weighted Rating: 1.8997183901551025
Title: 뮤지컬 〈쇼맨_어느 독재자의 네 번째 대역배우〉, Weighted Rating: 1.571623913466388
Title: 뮤지컬 〈오페라의 유령〉 - 서울, Weighted Rating: 2.089

,Title,Score
23,뮤지컬 〈타오르는 어둠 속에서〉,3.175342
33,뮤지컬 ＇벤허＇,3.055867
8,뮤지컬 〈렛미플라이〉,2.995440
10,뮤지컬 〈블랙메리포핀스〉,2.696585
2,뮤지컬 〈22년 2개월〉,2.555312


In [ ]:
title = "뮤지컬 〈칠칠〉"
musical_score(title).head(5)